## Kmeans implementation from scratch 

In [190]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.metrics.cluster import contingency_matrix

class KMeans:
    def __init__(self, k=10, max_iters=100):
        self.k = k
        self.max_iters = max_iters
        self.centroids = None
        self.labels = None
        self.objective_values = []  # Storing the objective function values

    def initialize_centroids(self, data):
        """
        Randomly initialising the starting position of the centroid
        """
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        return data[indices]

    def cluster_assignment(self, data, centroids):
        """
        Calculating the distance from data points to the closest centroid
        """
        # E-step
        distances = cdist(data, centroids, metric='euclidean')
        return np.argmin(distances, axis=1)

    def update_centroids(self, data, labels):
        """
        Recomputing the new position of centroids after revaluating the mean and variances
        """
        # M-step
        centroids = np.array([data[labels == i].mean(axis=0) for i in range(self.k)])
        return centroids
    
    def objective(self, data):
        """
        Function to calculate the kmeans objective
        """
        distances = cdist(data, self.centroids, 'euclidean')
        min_distances = np.min(distances, axis=1)
        return np.sum(min_distances ** 2)
    

    def fit(self, data):
        self.centroids = self.initialize_centroids(data)
        for i in range(self.max_iters):
            self.labels = self.cluster_assignment(data, self.centroids)
            new_centroids = self.update_centroids(data, self.labels)
            
            # Calculating the objective function after updating centroids
            objective_value = self.objective(data)
            self.objective_values.append(objective_value)
            
            # Check for convergence: if centroids don't change, terminate the loop
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids
            

    
# Functions to calculate the purity score and Gini index (clustering evaluation)
def purity_score(true_labels, centroid_labels):
    # Using contingency matrix for evaluating clustering accuracy
    contingency = contingency_matrix(true_labels, centroid_labels)
    majority = np.sum(np.amax(contingency, axis=0))
    purity = majority / np.sum(contingency)
    return purity


def gini_index(true_labels, centroid_labels):
    contingency = contingency_matrix(true_labels, centroid_labels)
    n_samples = np.sum(contingency)
    gini = 0

    for cluster in range(contingency.shape[1]):
        cluster_size = np.sum(contingency[:, cluster])
        if cluster_size == 0:
            continue

        prob = contingency[:, cluster] / cluster_size

        # Using a count for gini impurity
        count = 1 - np.sum(prob ** 2)

        # Updating gini index
        gini += (cluster_size / n_samples) * count
    return gini



# MNIST dataset 

In [113]:
from tensorflow.keras.datasets import mnist, fashion_mnist

# MNIST from keras
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Flatten/normalize the images for clustering
flattened_images = train_images.reshape((train_images.shape[0], -1))

# Initializing and fitting KMeans model
kmeans = KMeans(k=10, max_iters=300)
kmeans.fit(flattened_images)

# Calculate purity and Gini index
purity = purity_score(train_labels, kmeans.labels)
print(f"Purity Score: {purity:.5f}")

gini_index_score = gini_index(train_labels, kmeans.labels)
print(f"Gini Index: {gini_index_score:.5f}")

Purity Score: 0.59248
Gini Index: 0.54667


## FASHION Dataset

In [115]:
# FASHION MNIST dataset from keras
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Flatten the images for clustering
flattened_images = train_images.reshape((train_images.shape[0], -1))

# Initialize and fit KMeans model
kmeans = KMeans(k=10, max_iters=300)
kmeans.fit(flattened_images)

# Calculate purity and Gini index
purity = purity_score(train_labels, kmeans.labels)
print(f"Purity Score: {purity:.5f}")

gini_index_score = gini_index(train_labels, kmeans.labels)
print(f"Gini Index: {gini_index_score:.5f}")

Purity Score: 0.61380
Gini Index: 0.51725


## 20 NG Dataset

In [192]:
from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer


newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

# Fit and transform the dataset
X = vectorizer.fit_transform(newsgroups.data)    

# Converting sparse matrix to dense matrix
X_dense = X.toarray()

### Calculating purity and Gini index

In [185]:
#Calculating purity and Gini index
purity = purity_score(y, kmeans.labels)
gini = gini_index(y, kmeans.labels)

print(f'Purity Score: {purity}')
print(f'Gini Index: {gini}')

Purity Score: 0.515106303618053
Gini Index: 0.5621106918442331


## Gaussian Mixture Model to return the weights, mean and covariances from generated data

In [125]:
import numpy as np
from scipy.stats import multivariate_normal

# def load_data(file_path):
#     return np.loadtxt(file_path)

def initialize_parameters(data, num_components):
    """
    Randomly initialize means, covariances and weights
    :param data:
    :param num_components:
    :return:
    """
    np.random.seed(42)
    means = data[np.random.choice(data.shape[0], num_components, replace=False)]
    covariance = [np.cov(data, rowvar=False)] * num_components
    
    # Initialize weights evenly, assuming each component has an equal prior probability.
    # Weight signifies the probability that the data point was produced by a gaussian
    weights = np.ones(num_components) / num_components
    return means, covariance, weights

def e_step(data, weights, means, covariances, num_components):
    """
    Calculating the responsibilities/prior values for the initialised mean and covariances
    :param data:
    :param weights:
    :param means:
    :param covariance:
    :param num_components:
    :return:
    """
    # Array to hold the responsibilities, which is the probability of each each data point belonging to a particular gaussian
    prior_pi = np.zeros((data.shape[0], num_components))

    for i in range(num_components):
        # Calculate responsibilities by multiplying the p.d. of each data point for each Gaussian component by the component's weight.
        prior_pi[:, i] = weights[i] * multivariate_normal.pdf(data, mean=means[i], cov=covariances[i])
    
    # Normalize the responsibilities so that they sum to 1 for each data point across all the components.
    prior_pi /= prior_pi.sum(1)[:, np.newaxis]
    return prior_pi

def m_step(data, prior_pi, num_components):
    """
    Updating the weights, means and covariances based on the new responsibilities/priors
    :param data:
    :param prior_pi:
    :param num_components:
    :return:
    """
    n_k = prior_pi.sum(axis=0)
    weights = n_k / data.shape[0]
    means = np.dot(prior_pi.T, data) / n_k[:, np.newaxis]
    covariance = []

    for i in range(num_components):
        diff_mat = data - means[i]
        weighted_diff = diff_mat.T * prior_pi[:, i]
        covariance.append(np.dot(weighted_diff, diff_mat) / n_k[i])

    return weights, means, covariance

def gmm_step(data, num_components, max_iters=100):
    """
    Combining the EM steps
    :param data:
    :param num_components:
    :param max_iters:
    :return:
    """
    means, covariances, weights = initialize_parameters(data, num_components)

    for iteration in range(max_iters):
        prior_pi = e_step(data, weights, means, covariances, num_components)
        weights, means, covariances = m_step(data, prior_pi, num_components)

    return weights, means, covariances


In [124]:
# Loading datasets
data_2gauss = np.loadtxt('2gaussian.txt')

# GMM for 2 components
weights_2gauss, means_2gauss, covs_2gauss = gmm_step(data_2gauss, 2)
print("Estimated parameters for 2 gaussian mixtures:")
print("Weights:", weights_2gauss)
print("Means:", means_2gauss)
print("Covariances:", covs_2gauss)

# GMM for 3 components
# weights_3gauss, means_3gauss, covs_3gauss = gmm_em(data_3gauss, 3)
# print("\nEstimated parameters for 3 gaussian mixtures:")
# print("Weights:", weights_3gauss)
# print("Means:", means_3gauss)
# print("Covariances:", covs_3gauss)

Estimated parameters for 2 gaussian mixtures:
Weights: [0.33479577 0.66520423]
Means: [[2.99413182 3.0520966 ]
 [7.01314831 3.98313418]]
Covariances: [array([[1.01023424, 0.02719139],
       [0.02719139, 2.93782297]]), array([[0.97475893, 0.49747031],
       [0.49747031, 1.0011426 ]])]


## GMM implementation building the multivariate gaussian function from scratch

In [119]:
import numpy as np
from scipy.stats import multivariate_normal


def multivariate_gaussian(x, mean, cov):
    """
    Function to calculate the multivariate gaussian pdf
    :param x:
    :param mean:
    :param cov:
    :return:
    """
    k = len(mean)
    denom = np.sqrt(((2 * np.pi) ** k) * np.linalg.det(cov))
    diff = x - mean
    numerator = np.exp(-0.5 * np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff))
    gaussian = numerator / denom
    return gaussian


def initialize_parameters(data, num_components):
    """
    Randomly initialize means, covariances and weights
    :param data:
    :param num_components:
    :return:
    """
    np.random.seed(42)
    means = data[np.random.choice(data.shape[0], num_components, replace=False)]
    covariance = [np.cov(data, rowvar=False)] * num_components

    # Initialize weights evenly, assuming each component has an equal prior probability.
    weights = np.ones(num_components) / num_components
    return means, covariance, weights


def e_step(data, weights, means, covariance, num_components):
    """
    Calculating the responsibilities/prior values for the initialised mean and covariances
    :param data:
    :param weights:
    :param means:
    :param covariance:
    :param num_components:
    :return:
    """
    prior_pi = np.zeros((data.shape[0], num_components))

    for i in range(num_components):
        for point in range(data.shape[0]):
            prior_pi[point, i] = weights[i] * multivariate_gaussian(data[point], means[i], covariance[i])

    prior_pi /= prior_pi.sum(1)[:, np.newaxis]
    return prior_pi


def m_step(data, prior_pi, num_components):
    """
    Updating the weights, means and covariances based on the new responsibilities/priors
    :param data:
    :param prior_pi:
    :param num_components:
    :return:
    """
    n_k = prior_pi.sum(axis=0)
    weights = n_k / data.shape[0]
    means = np.dot(prior_pi.T, data) / n_k[:, np.newaxis]
    covariance = []

    for i in range(num_components):
        diff_mat = data - means[i]
        weighted_diff = diff_mat.T * prior_pi[:, i]
        covariance.append(np.dot(weighted_diff, diff_mat) / n_k[i])

    return weights, means, covariance


def gmm_step(data, num_components, max_iters=100):
    """
    Combining the EM steps
    :param data:
    :param num_components:
    :param max_iters:
    :return:
    """
    means, covariances, weights = initialize_parameters(data, num_components)

    for iteration in range(max_iters):
        prior_pi = e_step(data, weights, means, covariances, num_components)
        weights, means, covariances = m_step(data, prior_pi, num_components)

    return weights, means, covariances



## GMM for 2 gaussian mixtures

In [118]:
# Loading data from text file to numpy array
gauss_data = np.loadtxt('2gaussian.txt')


# GMM for 2 components
gauss_weights_2, gauss_mean_2, gauss_cov_2 = gmm_step(gauss_data, 2)
print("Estimated parameters for 2 gaussian mixtures:")
print("Weights:", gauss_weights_2)
print("Means:", gauss_mean_2)
print("Covariances:", gauss_cov_2)

Estimated parameters for 2 gaussian mixtures:
Weights: [0.33479577 0.66520423]
Means: [[2.99413182 3.0520966 ]
 [7.01314831 3.98313418]]
Covariances: [array([[1.01023424, 0.02719139],
       [0.02719139, 2.93782297]]), array([[0.97475893, 0.49747031],
       [0.49747031, 1.0011426 ]])]


## GMM for 3 gaussian mixtures

In [120]:
# Loading data from text file to numpy array
gauss_data = np.loadtxt('3gaussian.txt')

# GMM for 3 components
gauss_weights_3, gauss_mean_3, gauss_cov_3 = gmm_step(gauss_data, 3)
print("\nEstimated parameters for 3 gaussian mixtures:")
print("Weights:", gauss_weights_3)
print("Means:", gauss_mean_3)
print("Covariances:", gauss_cov_3)


Estimated parameters for 3 gaussian mixtures:
Weights: [0.49594551 0.20562097 0.29843352]
Means: [[5.01177664 7.00151504]
 [3.03980012 3.04879137]
 [7.02158525 4.01547353]]
Covariances: [array([[0.97965466, 0.18512102],
       [0.18512102, 0.97448689]]), array([[1.02858546, 0.02702432],
       [0.02702432, 3.38530379]]), array([[0.99037288, 0.50094401],
       [0.50094401, 0.99564696]])]


## EM algorithm to determine the biased and mixing probability of coin flips

In [189]:
import numpy as np
from scipy.stats import binom

def compute_responsibilities(data, num_flips, bias, prior):
    """
    Function to compute the responsibilities of the coin assignments
    given the current parameter estimates.
    :param data: data array
    :param bias: bias probabilities (P)
    :param prior: prior probabilities (P_i)
    :param num_flips: D
    :return:
    """
    # E-step
    num_heads = np.sum(data, axis=1, keepdims=True)
    likelihoods = binom.pmf(num_heads, num_flips, bias)
    weighted_likelihoods = likelihoods * prior
    normalized_likelihoods = weighted_likelihoods / np.sum(weighted_likelihoods, axis=1, keepdims=True)
    return normalized_likelihoods

def update_parameters(data, num_flips, responsibilities):
    """
    Function to update the P and P_i based on newly computed responsibilities
    """
    # M-step
    num_heads = np.sum(data, axis=1)
    updated_biases = np.dot(responsibilities.T, num_heads) / (num_flips * np.sum(responsibilities, axis=0))
    updated_priors = np.mean(responsibilities, axis=0)
    return updated_biases, updated_priors

def log_likelihood(data, num_flips, bias, prior):
    """
    Function to compute the log likelihood
    """
    num_heads = np.sum(data, axis=1, keepdims=True)
    likelihoods = binom.pmf(num_heads, num_flips, bias)
    weighted_likelihoods = likelihoods * prior
    total_likelihood = np.sum(weighted_likelihoods, axis=1)
    return np.sum(np.log(total_likelihood))

def coin_probabilities(data, num_flips, num_coins, max_iter=100, tolerance=1e-4):
    """
    Implementing the EM algorithm for coin toss
    """
    biases = np.random.rand(num_coins)
    priors = np.array([0.33,0.33,0.33])
    previous_log_likelihood = -np.inf

    for i in range(max_iter):
        responsibilities = compute_responsibilities(data, num_flips, biases, priors)
        biases, priors = update_parameters(data, num_flips, responsibilities)
        current_log_likelihood = log_likelihood(data, num_flips, biases, priors)

        if np.abs(current_log_likelihood - previous_log_likelihood) < tolerance:
            break
        previous_log_likelihood = current_log_likelihood

    return biases, priors

# Load the coin flip outcomes from the file
data = np.loadtxt('coin_flips_outcome.txt', dtype=int)

# Running the EM algorithm for coin flips
P, P_i = coin_probabilities(data, num_flips=20, num_coins=3)
print("Estimated bias probabilities:", P)
print("Estimated mixing probabilities:", P_i)


Estimated bias probabilities: [0.93172067 0.23670506 0.60986195]
Estimated mixing probabilities: [0.17858302 0.30641388 0.5150031 ]
